## Horses vs Humans Classifier 

Building the notebook from scratch 
  - using colab for this assigment because training the model without the GPU take a painfully long amount of time 

### Extracting data 
  Pull data from the data repository
  Extract the data 


### Split it train and validation 
  Split the data into train and validation sets, also have these in directories so that the generator can be leveraged directly 


### Create Generators
  After spliting the data we can create Generators for both train and validation 


### Define the model Architecture 
  Create a model architecture combinig convolution and Dense Layers

In [ ]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip 

In [ ]:
!unzip horse-or-human.zip
!mkdir all_data
!cp -r horses/ ./all_data/horses
!cp -r humans/ ./all_data/humans
!rm -r horses
!rm -r humans

In [ ]:
# For spliting the files, let's use os.listdir to get the list of files 
# planning ot create an 80 20 split so using numpy for that 
# and shutil to copy the files 
# Start by checking the number of files 


In [ ]:
import os 
import numpy as np 
import shutil

In [ ]:
humans = os.listdir('./all_data/humans/')
print('# of humans in the data: ',len(humans))
horses = os.listdir('./all_data/horses/')
print('# of horses in the data: ',len(horses))



In [ ]:
### There are approximately 500 images so we can use 400 for training and the remaing for testing 

human_prob = (np.random.rand(len(humans))>0.2)
horse_prob = (np.random.rand(len(horses))>0.2)
print("# of Humans in train: ",sum(human_prob))
print("# of Horses in train: ",sum(horse_prob))

In [ ]:
# Creating directories to be used by Humans 

os.mkdir('all_data/train/')
os.mkdir('all_data/validate/')
os.mkdir('all_data/train/humans/')
os.mkdir('all_data/train/horses/')
os.mkdir('all_data/validate/humans/')
os.mkdir('all_data/validate/horses/')


In [ ]:
# Splitting the data as train and test
for index,status in enumerate(human_prob):
  if status:
    shutil.copy('./all_data/humans/'+humans[index],'./all_data/train/humans/'+humans[index])
  else:
    shutil.copy('./all_data/humans/'+humans[index],'./all_data/validate/humans/'+humans[index])

for index,status in enumerate(horse_prob):
  if status:
    shutil.copy('./all_data/horses/'+horses[index],'./all_data/train/humans/'+horses[index])
  else:
    shutil.copy('./all_data/horses/'+horses[index],'./all_data/validate/humans/'+horses[index])

## Files are copied

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255)
validate_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_data = train_gen.flow_from_directory('all_data/train', #  Specifying the path of the directory that contains both the classes
                                           target_size=(300,300),
                                           batch_size=64,
                                           class_mode='binary')

In [ ]:
validate_data = validate_gen.flow_from_directory('all_data/validate', #  Specifying the path of the directory that contains both the classes
                                           target_size=(300,300),
                                           batch_size=16,
                                           class_mode='binary')

In [ ]:
from tensorflow import keras

In [ ]:
hor_hum_classifier = keras.Sequential([keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
                                       keras.layers.MaxPooling2D(2,2),
                                       keras.layers.Conv2D(32,(3,3),activation='relu'),
                                       keras.layers.MaxPooling2D(2,2),
                                       keras.layers.Conv2D(64,(3,3),activation='relu'),
                                       keras.layers.MaxPooling2D(2,2),
                                       keras.layers.Flatten(),
                                       keras.layers.Dense(512,activation='relu'),
                                       keras.layers.Dense(1,activation='sigmoid')])

In [ ]:
hor_hum_classifier.summary()

In [ ]:
hor_hum_classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
hor_hum_classifier.fit_generator(train_data, steps_per_epoch=13,
                                 validation_data=validate_data, epochs= 20,
                                 validation_steps= 4)


In [56]:
## We have a classifier with almost 100 % accuracy to test it on original data use the image class from keras preprcessing